# Cleaning duplicates based on src, dest, timestamp
it can work on all types or just one 
it finds all the combinations of sources and destinations and does them one by one

In [1]:
from datetime import datetime
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan
from elasticsearch.helpers import bulk
es = Elasticsearch(['atlas-kibana.mwt2.org:9200'],timeout=60)

In [2]:
ind = [
    "network_weather-2017.1.*","network_weather-2017.2.*","network_weather-2017.3.*",
    "network_weather-2017.4.*","network_weather-2017.5.*"
]

my_query = \
{
  "size": 0,
  "aggs": {      
        "type_counts": {
            "terms": { "field": "_type" },
            "aggs": {      
                "src_counts": {
                     "terms": { "field": "src" , "size": 10000},
                     "aggs": {      
                        "dest_counts": {
                        "terms": { "field": "dest" , "size": 10000}
                        }
                     }
                  }
              }
        }
  }
}
response = es.search(index=ind, body=my_query, request_timeout=1200)
#print(response['aggregations']['type_counts']['buckets'])

In [4]:
# for cleaning specific links
limitTo=['192.170.227.160']#,'192.170.227.163']

skipDest=["149.165.225.223","129.93.5.165","149.165.225.3","149.165.225.1","149.165.224.247","129.107.255.26",
          "192.41.230.61","192.170.227.163","192.12.15.111","192.170.230.51","192.41.236.31","192.5.207.251",
          "157.82.112.68","141.108.35.18","144.206.237.142","134.158.103.10","192.108.47.12","134.158.123.183",
          "141.34.200.28","195.194.105.178","163.1.5.210","192.231.127.41","117.103.105.191","129.107.255.29",
          "194.190.165.192","192.54.207.250","145.100.17.8","147.231.25.192","130.246.176.109","192.114.101.125",
          "147.156.116.40","144.206.236.189","194.36.11.38","142.150.19.61","131.111.66.196","128.142.223.247",
          "194.80.35.169","148.187.64.25","138.253.60.82","134.158.73.243","134.158.159.85","134.79.118.72",
          "158.195.14.26","129.15.40.231","194.85.69.75","193.48.99.76","134.158.132.200","131.154.254.12",
          "206.12.9.2"]

In [5]:
types = response['aggregations']['type_counts']['buckets']

for t in types:
    
    # if not t['key']=='traceroute': continue
    if not t['key']=='packet_loss_rate': continue
    print(t['key'])
    
    allPairs=0
    for s in t['src_counts']['buckets']:
        
        src=s['key']
        if src not in limitTo: continue
        
        bulk_body = []
        
        for d in s['dest_counts']['buckets']:
            
            dest=d['key']
            
           # if dest not in limitTo: continue
            if dest in skipDest: continue
                
            toDelete={}
            allPairs += 1
            
            print ('s: ',src,'d: ', dest, '\t docs:', d['doc_count'])
            
            myscroll={
                "size":0,
                "_source": {
                    "includes": [ "timestamp" ]
                },
#               "fields": ["timestamp", "delay_mean","throughput","packet_loss","src","dest"],
                "query":{
                    "bool" : {
                        "must" : [
                            {"term": {"_type": {"value": t['key'] } } },
                            {"term": {"src":   {"value": src } } },
                            {"term": {"dest":  {"value": dest } } }
                        ]
                    }
                }
            #,"sort": [{ "timestamp": { "order": "asc", "unmapped_type": "boolean" }}]
            }
            
            times=[]
            counter=0
            scroll = scan(es, query=myscroll, index=ind, size=5000, scroll='240s', request_timeout=120)
            for res in scroll:
                if not counter%100000: print(counter)
                counter+=1
    
                ts=res['_source']['timestamp']
                if ts in times:# and res['fields']['src']==src and res['fields']['dest'][0]==dest and res['fields']['packet_loss'][0]==oldres:
                    toDelete[res['_id']]=res['_index']
#                     print(res['fields'])
                else:
                    times.append(ts)
#                if counter>200:
#                    break
#                 if counter%100==0: 

            print ("records:", counter, '\tto delete:',len(toDelete))
            dcounter=0
            for x in toDelete:
                if dcounter>100000:
                    dcounter=0
                    es.bulk('\n'.join(bulk_body), request_timeout=1200)
                    bulk_body=[]
                    print('deleted 100k')
                dcounter+=1
                bulk_body.append('{"delete": {"_id": "'+x+'", "_index":"'+toDelete[x]+'", "_type":"'+t['key']+'"}}')
#             print (bulk_body)
            if dcounter:
                print("final delete", dcounter)
                es.bulk('\n'.join(bulk_body), request_timeout=1200)

    print(t['key'],'Pairs:',allPairs)
print ('DONE.')

packet_loss_rate
s:  192.170.227.160 d:  72.36.96.4 	 docs: 271254
0
100000
200000
records: 271254 	to delete: 120
final delete 120
s:  192.170.227.160 d:  193.48.83.165 	 docs: 74806
0
records: 74823 	to delete: 0
s:  192.170.227.160 d:  131.169.98.30 	 docs: 74612
0
records: 74629 	to delete: 0
s:  192.170.227.160 d:  146.83.90.7 	 docs: 74558
0
records: 74571 	to delete: 0
s:  192.170.227.160 d:  134.158.20.192 	 docs: 74290
0
records: 74307 	to delete: 0
s:  192.170.227.160 d:  130.209.239.124 	 docs: 74271
0
records: 74288 	to delete: 0
s:  192.170.227.160 d:  202.122.32.170 	 docs: 66852
0
records: 66869 	to delete: 0
s:  192.170.227.160 d:  192.12.15.26 	 docs: 66086
0
records: 66114 	to delete: 0
s:  192.170.227.160 d:  206.12.24.251 	 docs: 65724
0
records: 65741 	to delete: 0
s:  192.170.227.160 d:  130.246.47.129 	 docs: 65508
0
records: 65526 	to delete: 0
s:  192.170.227.160 d:  85.122.31.74 	 docs: 64277
0
records: 64294 	to delete: 0
s:  192.170.227.160 d:  193.109.172.1